## **MA5606 Tópicos Matemáticos en Aprendizaje de Máquinas, Redes Neuronales y Aprendizaje Profundo**

### **Tarea 1: Redes neuronales feedfoward y PINNs**

**Profesores: Claudio Muñoz y Joaquín Fontbona**

**Auxiliares: Javier Maass y Diego Olguín**

**Nombres integrantes: COMPLETAR**

**Instrucciones:**

- **Fecha de entrega:** **26 de abril de 2024, a las 23:59.**

- **Importante:** Si trabaja desde el link de Google Colab (muy recomendable para trabajar con DeepXDE) debe hacer un copia en su Drive antes de trabajar, de lo contrario se podrían no guardar sus códigos.

- Debe entregar un Jupyter Notebook (archivo .ipynb) con sus código en Python. Le pueden ser de mucha utilidad los códigos vistos en la actividad práctica.

- Sus códigos deben estar comentados y ordenados. Además, en formato texto debe colocar todas sus conclusiones y resultados pedidos que deban ser redactados.

- En todos los ejercicios se le pide hacer al menos un gráfico. Los gráficos que realicen deben ser claros, con títulos y nombres en los ejes, junto con leyendas si es que corresponde.

In [1]:
# Librerías

# Numpy y matplotlib, junto con seaborn, para gráficos un poco mejores
import numpy as np
import matplotlib.pyplot as plt

# Puede ser útil para hacer gráficos con barras de colores
from mpl_toolkits.axes_grid1 import make_axes_locatable

import seaborn as sns

sns.set_theme()

# PyTorch y módulos que serán necesario
import torch
import torch.nn as nn
import torch.optim as optim

## **Ejercicio 1**

El objetivo de este ejercicio es estudiar la aproximación vía redes neuronales del problema de frontera

$$ y''(x) + \frac{\pi^2}{4} y(x) = 0 $$
$$ y(-1) = y(1) = 0, \, y(0) = 1$$

Para ello utilice redes de 1 capa oculta de ancho $N$, usando $N = \{ 10, 20, 30, 100\}$. Entrene la red con 600 iteraciones del algoritmo ``Adam``.

En cada caso grafique la evolución de la función de pérdida en las iteraciones de entrenamiento, además de la red evaluada en puntos en el intervalo $[-1,1]$ y compare con la solución analítica:

$$ y(x) = \cos \left (\frac{\pi}{2} x \right ) $$

In [3]:
# Creo la clase que define la red neuronal con el codigo del auxiliar:

class NeuralNetwork(nn.Module):
    def __init__(self, dim_input, dim_output, n_layers, layer_width, activation) -> None:
        super().__init__()
        # Se crean las capas internas
        self.inner_layers = nn.ModuleList([nn.Linear(dim_input if n == 0 else layer_width, layer_width)
                                           for n in range(n_layers - 1)])
        # Y la última capa, o de output
        self.last_layer = nn.Linear(layer_width, dim_output)
        self.activation = activation

    # Función de forward, que dado un input calcula el output de la red
    def forward(self, input):

        for layer in self.inner_layers:
            input = self.activation()(layer(input)) # Efecto de activación

        return self.last_layer(input)

In [16]:
# Me robo también el código para las derivadas

def derivative(f, variable):
    return torch.autograd.grad(f.sum(), variable, create_graph=True)[0]

# Finalmente algo que escribo yo:

def ode_difference(x, net):
    y = net(x).unsqueeze(0)
    net_y = y[:, 0]
    net_dy = y[:, 1]
    dy = derivative(y, x)
    dnet_dy = dy[:, 0]
    dnet_ddy = dy[:, 1]
    print(y.shape)
    print(dy.shape)
    print(x.shape)

    return 

n_layers = 3 # Número de capas
wide = 100 # Ancho

# Dimensiones de entrada y salida, que serán iguales en este caso
dim_input = 1
dim_output = 1

# Red neuronal
net = NeuralNetwork(dim_input, dim_output, n_layers, wide, nn.Tanh)

# Optimizador
optimizer = torch.optim.Adam(net.parameters())

# Pesos de las funciones de pérdida
weights = (1, 1)

# Valor inicial
y_initial = 1/2

# Iteraciones de entrenamiento
iters = 600

# Discretización en tiempo
samples_phys = 300

# Iremos guardando la pérdida en cada época
loss_record = []


# Posiciones en x a evauar
x_initial = torch.zeros(1, 1)

# Tiempo para evaluar lo que devuelve el residuo de la EDO
x_phys = torch.linspace(0, 2, samples_phys).reshape((samples_phys, 1)).requires_grad_()

def calc_loss():

    ####
    # Pérdida en la condición inicial
    ####

    # Predicción en el medio
    output_initial = net(x_initial).unsqueeze(0)

    # Objetivo o target, es lo que debería dar, en este caso
    # la condición inicial real
    target_initial = torch.ones(1, 1, 1)*y_initial

    # Cálculo de la perdida en la condición inicial
    loss_initial = nn.MSELoss()(output_initial, target_initial)

    ####
    # Pérdida en la EDO o la física
    ####

    # Evaluación de la red en el residuo de la EDO
    output_phys = ode_difference(x_phys, net)

    # Objetivo, nos debería dar 0 el residuo
    target_phys = torch.zeros_like(output_phys)

    # Cálculo de la perdida en el residuo de la EDO
    loss_phys = nn.MSELoss()(output_phys, target_phys)

    ####
    # Pérdida total
    ####

    # Cálculo de la función de pérdida total
    loss =  weights[0]*loss_phys + weights[1]*loss_initial

    return loss

# Función necesaria para el optimizador
def closure():
    optimizer.zero_grad() # Se actualizan los pesos haciendo gradiente 0
    loss = calc_loss() # Se calcula la función de pérdida
    loss.backward() # Diferenciación automática
    return loss

try:
    for iter in range(1, iters + 1):

        # Guardamos la pérdida calculada
        loss_record.append(calc_loss().item())
        # Paso del optimizador
        optimizer.step(closure)

        # Imprimimos la pérdida en cada iteración, cada ciertas iteraciones
        if iter % 100 == 0:
            print(f'Iterations {iter} || loss: {loss_record[-1]:.4f}')

# Hacer esta sentencia try, except de esta forma hace que
# podamos parar el entrenamiento sin que haya error
except KeyboardInterrupt:
    pass

torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size([1, 300, 1])
torch.Size([300, 1])
torch.Size([300, 1])
torch.Size

## **Ejercicio 2**



Veremos nuevamente el problema de EDP de Helmholtz

$$ -\Delta u - k_0^2 u = f, \quad \Omega = (0,1)^2 $$
$$ u = 0, \quad ∂ Ω $$

Que tiene solución analítica

$$ u(x, y) =  \sin (k_0 x) \sin (k_0 y)$$

Cuando $f(x,y) = k_0^2 \sin (k_0 x) \sin (k_0 y)$. Considere $k_0 = 2 n \pi$ y entrene una red neuronal de 3 capas, 100 neuronas por capa, función de activación seno y 3000 iteraciones del optimizador Adam, que resuelva el problema para $n \in \{ 1, 3, 5 \}$. Grafique la función de pérdida en función de las iteraciones de entrenamiento y el resultado de la red, comparando este con la solución analítica. ¿Qué resultado observa para los distintos $n$ propuestos?

In [2]:
# Instalamos e importamos la librería
try:
    import deepxde as dde
except:
    !pip install deepxde
    import deepxde as dde

# Importamos Tensorflow, que es el backend que utilizaremos
import tensorflow as tf

2024-04-10 16:49:44.146570: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 16:49:44.205527: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 16:49:44.415740: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 16:49:45.125125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using backend: tensorflow
Other supported backends: tensorflow.compat.v1, pytorch, jax, paddle.
paddle supports more 

In [ ]:
# Solución!

### **Ejercicio 3**

Considere el problema de Poisson

$$ -\Delta u = x(1-x) + y(1-y), \quad \Omega = (0,1)^2 $$
$$ u = 0, \quad ∂ Ω $$

Que tiene solución analítica

$$ u(x, y) = \frac{1}{2} x(1-x)y(1-y)$$

Entrene una red neuronal que aproxime la solución al problema, para ello considere una red de 3 capas, 100 neuronas por cada, función de activatión tangente hiperbólica y 500 iteraciones del optimizador Adam. Grafique la función de pérdida en función de las iteraciones de entrenamiento y el resultado de la red, comparando este con la solución analítica del problema.


In [ ]:
# Solución!